In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
a = pd.read_csv("/content/drive/MyDrive/專題資料夾/程式碼的家/Finance_data/大盤股價日資料.csv")

UnicodeDecodeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') # 此處需要登入 google 帳號


Mounted at /content/gdrive


In [ ]:
import pandas as pd
import ast
import re
import jieba
import string
import numpy
import pickle

In [ ]:
with open('/content/drive/MyDrive/專題資料夾/程式碼的家/pkl檔案/盤後.pkl', 'rb') as f:
    data = pickle.load(f)
dfA = pd.DataFrame(data)

with open('/content/drive/MyDrive/專題資料夾/程式碼的家/pkl檔案/盤中.pkl', 'rb') as f:
    data1 = pickle.load(f)
dfD = pd.DataFrame(data1)
OP = pd.read_excel('/content/drive/MyDrive/專題資料夾/程式碼的家/opinion.xlsx')


In [ ]:
OP['情緒分數'].sum()

-622.7204120000001

In [ ]:
def push_sentiment_calculator(Pushes):
  #data_str = Pushes
  #print(data_str)
  data = Pushes
  contents = [item['Content'] for item in data if 'Content' in item and not re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', item['Content'])]
  return contents

all_list_dfA = [] # 最後會是二維的，天數跟當天個別作者留言
for i in range(len(dfA['Pushes'])):
  temp = push_sentiment_calculator(dfA['Pushes'][i])
  all_list_dfA.append(temp)

all_list_1da = []
for sublist in all_list_dfA:
    for element in sublist:
        all_list_1da.append(element)

dfAsentance_cutted = []

for i in range(len(all_list_1da)):
  #each_sentance_and_word = []
  each_sentance_and_word = list(jieba.cut(all_list_1da[i]))
  dfAsentance_cutted.append(each_sentance_and_word)

all_list_dfD = [] # 最後會是二維的，天數跟當天個別作者留言
for i in range(len(dfD['Pushes'])):
  temp = push_sentiment_calculator(dfD['Pushes'][i])
  all_list_dfD.append(temp)

all_list_1dd = []
for sublist in all_list_dfD:
    for element in sublist:
        all_list_1dd.append(element)

dfDsentance_cutted = []

for i in range(len(all_list_1dd)):
  #each_sentance_and_word = []
  each_sentance_and_word = list(jieba.cut(all_list_1dd[i]))
  dfDsentance_cutted.append(each_sentance_and_word)

sentance_cutted = dfAsentance_cutted + dfDsentance_cutted

In [ ]:
all_list_dfD[:2]

In [ ]:
import jieba

data = all_list_dfD
# 對第二個維度的字進行jieba切詞
cut_dataD = []
for sublist in data:
    cut_sublist = [list(jieba.cut(word)) for word in sublist]
    cut_dataD.append(cut_sublist)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(cut_dataD[:2])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
df = pd.DataFrame(cut_dataA)
output_path = '/content/drive/MyDrive/專題資料夾/程式碼的家/outputdfa.csv'
df.to_csv(output_path, index=False, header=False)

In [ ]:
df = pd.DataFrame(cut_dataD)
output_path = '/content/drive/MyDrive/專題資料夾/程式碼的家/outputdfd.csv'
df.to_csv(output_path, index=False, header=False)

In [ ]:
# output 切好的資料檔案
import json
output_path = '/content/drive/MyDrive/專題資料夾/程式碼的家/output.json'
with open(output_path, "w") as file:
    json.dump(sentance_cutted, file)

In [ ]:
# input 切好的資料檔案
input_path = '/content/drive/MyDrive/專題資料夾/程式碼的家/output.json'
with open(input_path, "r") as file:
    sentance_cutted = json.load(file)
print(sentance_cutted)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
len(sentance_cutted)

3610675

In [ ]:
word_count_dict = {}
my_2d_list = sentance_cutted
excluded_words = ['我', '你', ',', '！', '，','了','又','？','的','是','就','要','在','都','有','嗎','也','會',]
for sublist in my_2d_list:
    for word in sublist:
        word_cleaned = word.strip()
        if word_cleaned and word_cleaned not in string.punctuation and word_cleaned not in excluded_words:
            if word_cleaned in word_count_dict:
                word_count_dict[word_cleaned] += 1
            else:
                word_count_dict[word_cleaned] = 1

# 按照出现次数降序排序字典
sorted_word_count = sorted(word_count_dict.items(), key=lambda x: x[1], reverse=True)

# 打印前20个字符串及其出现次数
for i, (word, count) in enumerate(sorted_word_count[:200], 1):
    print(f'{i}. {word}: {count}')

In [ ]:
from gensim.models import Word2Vec

# 訓練詞向量模型
model = Word2Vec(sentance_cutted, vector_size = 50, window=5, min_count=1, workers=4)

# 獲取所有詞
vocab = model.wv.index_to_key

# 獲取所有詞向量
vectors = model.wv[vocab]

vectors

array([[-1.1841789e+00,  1.0908576e-01, -2.3723589e-01, ...,
         1.4289839e+00,  6.3050884e-01,  6.4054692e-01],
       [-2.2832725e+00,  3.2737665e+00, -1.1151643e+00, ...,
        -2.7268927e+00, -4.8697752e-01, -1.4124497e+00],
       [-1.5760039e-01,  4.0960976e-01,  7.3733175e-01, ...,
         1.2996760e-01, -9.6711284e-01, -9.6671301e-01],
       ...,
       [-2.7393434e-02, -3.4894966e-02, -1.2547653e-02, ...,
        -3.8515147e-02,  9.1235954e-03,  9.3889760e-04],
       [-1.8132465e-02, -2.2226507e-03,  1.7072236e-02, ...,
        -4.4379029e-02,  5.9032746e-02,  3.5718866e-02],
       [ 1.7483190e-02,  2.9720472e-02, -2.2589756e-02, ...,
        -2.1706587e-02,  5.4392084e-02,  7.4399011e-03]], dtype=float32)

In [ ]:
# output vectors
import numpy as np
output_path = "/content/drive/MyDrive/專題資料夾/程式碼的家/outputvectors.npy"
np.save(output_path, vectors)


NameError: ignored

In [ ]:
# input vectors
vectors = np.load(output_path)
print(vectors)

[[-1.1841789e+00  1.0908576e-01 -2.3723589e-01 ...  1.4289839e+00
   6.3050884e-01  6.4054692e-01]
 [-2.2832725e+00  3.2737665e+00 -1.1151643e+00 ... -2.7268927e+00
  -4.8697752e-01 -1.4124497e+00]
 [-1.5760039e-01  4.0960976e-01  7.3733175e-01 ...  1.2996760e-01
  -9.6711284e-01 -9.6671301e-01]
 ...
 [-2.7393434e-02 -3.4894966e-02 -1.2547653e-02 ... -3.8515147e-02
   9.1235954e-03  9.3889760e-04]
 [-1.8132465e-02 -2.2226507e-03  1.7072236e-02 ... -4.4379029e-02
   5.9032746e-02  3.5718866e-02]
 [ 1.7483190e-02  2.9720472e-02 -2.2589756e-02 ... -2.1706587e-02
   5.4392084e-02  7.4399011e-03]]


In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA

# 设置降维后的维度
reduced_dim = 4

# 使用PCA进行降维
pca = PCA(n_components=reduced_dim)
vectors_reduced = pca.fit_transform(vectors)

In [ ]:
import pandas as pd

# 给定的 DataFrame
data = OP
df_emotion = pd.DataFrame(data)

# 将 DataFrame 转换为字典
emotion_dict = df_emotion.set_index('情緒字詞')['情緒分數'].to_dict()


emotion_words = vocab
data = {'情緒字詞': emotion_words}
df_emotion = pd.DataFrame(data)

df_emotion['情緒分數'] = df_emotion['情緒字詞'].map(emotion_dict).fillna(0)
df_emotion

,情緒字詞,情緒分數
0,,0.0
1,了,0.0
2,的,0.0
3,，,0.0
4,是,0.0
...,...,...
621296,吐成,0.0
621297,幫換頁,0.0
621298,開賓利載,0.0
621299,換頁,0.0


In [ ]:
non_zero_rows = df_emotion[df_emotion['情緒分數'] != 0.0]
count_non_zero = non_zero_rows.shape[0]
#print("情緒分數不為0的行數:", count_non_zero)
#print(df_emotion)
vr = pd.DataFrame(vectors_reduced, columns=['X', 'Y','Z','A'])
#print(vr)
merged_df = pd.concat([df_emotion, vr], axis=1)
print(merged_df)

df = merged_df

# 拆分有情緒分數和沒有情緒分數的兩個DataFrame
df_with_score_zero = df[df['情緒分數'] == 0.0].reset_index(drop=True)
df_without_score_zero = df[df['情緒分數'] != 0.0].reset_index(drop=True)

print("DataFrame with score 0:")
print(df_with_score_zero)

print("\nDataFrame without score 0:")
print(df_without_score_zero)


        情緒字詞  情緒分數         X         Y         Z         A
0              0.0  3.384539  6.399894  0.622428 -0.721509
1          了   0.0  3.157820  7.232344  0.779885 -0.416625
2          的   0.0  5.680951  8.788043  3.988590 -2.451832
3          ，   0.0  4.801659  9.693619  1.807864 -0.563806
4          是   0.0  5.485770  7.605177  2.382926 -0.329187
...      ...   ...       ...       ...       ...       ...
621296    吐成   0.0 -0.155777  0.071558  0.094497  0.000869
621297   幫換頁   0.0 -0.185169  0.121484  0.068274 -0.001206
621298  開賓利載   0.0 -0.162412  0.033533  0.051935  0.002080
621299    換頁   0.0 -0.056598  0.001870  0.009919 -0.004869
621300   蘇飛到   0.0 -0.158934  0.017267  0.014081 -0.017369

[621301 rows x 6 columns]
DataFrame with score 0:
        情緒字詞  情緒分數         X         Y         Z         A
0              0.0  3.384539  6.399894  0.622428 -0.721509
1          了   0.0  3.157820  7.232344  0.779885 -0.416625
2          的   0.0  5.680951  8.788043  3.988590 -2.451832
3    

In [ ]:
df_with_score_zero_coor = df_with_score_zero[['X', 'Y','Z','A']].to_numpy()
df_without_score_zero_coor = df_without_score_zero[['X', 'Y','Z','A','情緒分數']].to_numpy()
print("沒有分數的")
print(df_with_score_zero_coor)
print("有分數的")
print(df_without_score_zero_coor)

沒有分數的
[[ 3.3845387e+00  6.3998942e+00  6.2242812e-01 -7.2150922e-01]
 [ 3.1578195e+00  7.2323437e+00  7.7988505e-01 -4.1662541e-01]
 [ 5.6809511e+00  8.7880430e+00  3.9885900e+00 -2.4518316e+00]
 ...
 [-1.6241163e-01  3.3532739e-02  5.1935453e-02  2.0796848e-03]
 [-5.6598190e-02  1.8704896e-03  9.9189775e-03 -4.8692808e-03]
 [-1.5893368e-01  1.7266745e-02  1.4080716e-02 -1.7368944e-02]]
有分數的
[[ 4.52966690e+00  6.85144711e+00  1.64390135e+00 -4.54489183e+00
   1.55180000e-01]
 [ 1.87857831e+00  5.93015480e+00  4.31996107e+00  1.62929352e-02
  -3.68710000e-01]
 [ 4.90639496e+00  6.94019032e+00  2.94925809e+00 -2.37274513e-01
  -7.60930000e-01]
 ...
 [-1.09095991e-01  2.53522731e-02  3.60301808e-02  1.60960127e-02
  -1.64500000e-02]
 [-1.71893194e-01  5.23330197e-02  5.10262996e-02 -4.76026647e-02
  -8.75000000e-01]
 [-1.14808790e-01  4.94595664e-03  3.07718273e-02 -2.46592797e-02
   2.18839000e-01]]


In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from multiprocessing import Pool

def calculate_distance_parallel(chunk):
    distances = []
    for point1 in chunk:
        ss = 1 / cdist([point1], points2, metric='minkowski', p=4)
        #print(ss)
        distance_row = 1 / cdist([point1], points2, metric='minkowski', p=4)
        a = np.sum(distance_row)
        distances.append(a)
    return distances

# 提取座標部分
points1 = df_with_score_zero_coor[:, :4]
points2 = df_without_score_zero_coor[:, :4]

# 將第一個表的數據拆分成多個區塊
num_chunks = 10
chunks = np.array_split(points1, num_chunks)

# 使用多進程進行平行計算
with Pool() as pool:
    results = pool.map(calculate_distance_parallel, chunks)

# 將結果合併
distances = np.concatenate(results)
print(distances)

[ 1194.51586765  1068.22866011   836.40800508 ... 30757.19930898
 36377.06144408 27971.91516384]


In [ ]:
output_path = "/content/drive/MyDrive/專題資料夾/程式碼的家/outputdistances.npy"
np.save(output_path, distances)

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from multiprocessing import Pool

def calculate_weighted_distance_parallel(chunk):
    weighted_distances = []
    distances1 = []
    for point1 in chunk:
        distances1 = 1 / cdist([point1], points2, metric='minkowski', p=4).flatten()  # 計算距離倒數
        #print(distances1)
        weighted_distance_row = np.dot(distances1, value)  # 與分數進行加權加總
        #print(weighted_distance_row)
        weighted_distances.append(weighted_distance_row)
    return weighted_distances


# 提取座標部分
points1 = df_with_score_zero_coor[:, :4]
points2 = df_without_score_zero_coor[:, :4]
value = df_without_score_zero_coor[:, -1]

# 將第一個表的數據拆分成多個區塊
num_chunks = 5  # 可根據實際情況調整
chunks = np.array_split(points1, num_chunks)

# 使用多進程進行平行計算
with Pool() as pool:
    results = pool.map(calculate_weighted_distance_parallel, chunks)

# 將結果合併
weighted_distances = np.concatenate(results)

# weighted_distances現在包含了第一個表每個點與第二個表所有點的距離加權加總
print(weighted_distances)
#print(value)

[  -26.96526039   -25.48394386   -20.53041493 ... -1844.05030911
 -2154.07418127 -1661.22857832]


In [ ]:
input_path = "/content/drive/MyDrive/專題資料夾/程式碼的家/outputdistances.npy"
distances = np.load(input_path)
print(distances)

[ 1194.51586765  1068.22866011   836.40800508 ... 30757.19930898
 36377.06144408 27971.91516384]


In [ ]:
output_path = "/content/drive/MyDrive/專題資料夾/程式碼的家/output_weighted_distances.npy"
np.save(output_path, weighted_distances)

In [ ]:
input_path = "/content/drive/MyDrive/專題資料夾/程式碼的家/output_weighted_distances.npy"
weighted_distances = np.load(input_path)
print(weighted_distances)

[  -26.96526039   -25.48394386   -20.53041493 ... -1844.05030911
 -2154.07418127 -1661.22857832]


In [ ]:
final_sentiment = np.divide(weighted_distances, distances)
print(final_sentiment)

[-0.02257422 -0.02385626 -0.02454593 ... -0.05995508 -0.05921518
 -0.05938916]


In [ ]:
df_with_score_zero['情緒分數'] = final_sentiment

In [ ]:
output_path = "/content/drive/MyDrive/專題資料夾/程式碼的家/NoScoredictionary.csv"
df_with_score_zero[['情緒字詞','情緒分數']].to_csv(output_path, index=False)

In [ ]:
import pandas as pd
dic = pd.read_csv('/content/drive/MyDrive/專題資料夾/程式碼的家/NoScoredictionary.csv')
dic

In [ ]:
df = df_with_score_zero
sorted_df = df.sort_values(by='情緒分數', ascending=True)
sorted_df[:50]

In [ ]:
all_list = [] # 最後會是二維的，天數跟當天個別作者留言
for i in range(20):
  temp = push_sentiment_calculator(dfA['Pushes'][i])
  all_list.append(temp)
print(all_list)

[[' 衝啊', ' 我斷 斷了思念', ' 外資…', ' 外資跑了 誰要洗碗', ' 今天應該沒人沒買吧', ' 查理的靈壓消失了嗚嗚嗚', ' 中', ' 法人大加空，嘻嘻', ' 推', ' 天天賠錢   沒心情斷推圖惹', ' 我懂', ' 華字輩的射飛鏢買都賺錢了', ' yo之前不是大賺嗎', ' 聾鴿今天有重播嗎？', ' 期指價差那麼多 夜盤可以大爆VVVVVVVVVVV了', ' Yo叔之前賺爛 不差小條的吧', ' (．w．) 中奶', ' 自營....又要搞事嗎', ' 華字輩的站出來', ' 港股該不會破16900吧', ' 丸子!外資唱雖台灣CC', ' 庫存每天減少的感覺大家應該都能感同身受吧', ' 自營又做空？', ' 恆生掰', ' 我想說yo叔怎麼沒來斷~', ' yo叔這兩天都沒什麼在閒聊耶', ' 無敵軋空手 VVVVVVVVV', ' Yo怎麼沒跑掉嗎', ' 跑四千口', ' 有人知道港股在跌什麼？', ' 剛剛心情差到直接拿吹又機去山上把農路樹葉吹乾淨', ' 連股市即時行情都不看惹', ' 阿文995', '                   吹葉機', ' 自營都不信能過前高了', ' 何止跑4328口  小台也跑了9257台  一定有什麼消息', ' yo叔的美晶有買回來嗎', ' 今天融卷應該又加空吧', ' 想再凹單 凹個11月營收 看能凹些回來嗎', ' 外資自營商聯手？', ' 港股跌 不代表台股會漲 哀衿勿喜', ' 沒買到耶  錢都在單隻股票 那天賣本來要進 結果起漲', ' 田僑仔的生活就是這麼樸實無華', ' 岳父怎麼了 不是大爆V嗎', ' 香港要滅亡了嗎', ' 後來又買回同支 結果美晶就噴出了  股市都是早知道', ' 法人打算留信貸蛙洗碗？', ' 相澤南<3<3<3<3<3<3<3<', ' 香港爆炸', ' 空爆', ' 美股沒崩 台股不要怕', ' 港股怎麼大爆炸啊', ' 外豬就把前幾天買的倒出來', ' 之前恐嚇人的垃圾id  現在變成狂問還有沒有人沒上', ' 車 要噴了', ' yo叔又歐印新建喔', ' 聰明的就趕快逢高調節拉高現金比例吧', ' 聾哥做空讓很安心', ' 丸子 小兒腳底抹油', ' 台幣繼續升欸', ' 奇怪 尾盤鬼指怎麼自己崩了？', ' 今天外資會買

In [ ]:
dic = pd.read_csv('/content/drive/MyDrive/專題資料夾/程式碼的家/NoScoredictionary.csv')
OP = pd.read_excel('/content/drive/MyDrive/專題資料夾/程式碼的家/opinion.xlsx')

In [ ]:
print(dic)
print(OP)

        情緒字詞      情緒分數
0            -0.022574
1          了 -0.023856
2          的 -0.024546
3          ， -0.024108
4          是 -0.022793
...      ...       ...
613712    吐成 -0.064530
613713   幫換頁 -0.061757
613714  開賓利載 -0.059955
613715    換頁 -0.059215
613716   蘇飛到 -0.059389

[613717 rows x 2 columns]
            情緒字詞      情緒分數
0          一下子爆發 -0.388750
1      一下子爆發的一連串 -0.208390
2           一夕成名  0.000000
3           一大打擊 -0.405990
4            一大步  0.048718
...          ...       ...
28794         鐐銬  0.000000
28795         礭實  0.381137
28796         飆出  0.019193
28797         攢動  0.000000
28798         躥起  0.000000

[28799 rows x 2 columns]


In [ ]:
total_dic = pd.concat([dic, OP], ignore_index=True)
total_dic

,情緒字詞,情緒分數
0,,-0.022574
1,了,-0.023856
2,的,-0.024546
3,，,-0.024108
4,是,-0.022793
...,...,...
642511,鐐銬,0.000000
642512,礭實,0.381137
642513,飆出,0.019193
642514,攢動,0.000000


In [ ]:
all_list_1dd